In [8]:
# Import the Data
import json
from pprint import pprint

#Read in complete dataset
with open('r_data/one_data_to_rule_them_all.json', 'r', encoding='utf-8') as file:
    complete_dataset = json.loads(file.read())

file.close()
#pprint(type(complete_dataset))
print("First submission element \n")
pprint(complete_dataset[0])

First submission element 

{'author': 'ZachityZach',
 'author_flair_text': None,
 'child': [{'author': 'tiki77747',
            'body': 'dude same here',
            'child': [{'author': 'ZachityZach',
                       'body': "Omfg tiki :3 I'm in your stream all the time "
                               'as ZakoLive',
                       'child': [{'author': 'tiki77747',
                                  'body': 'OHEY. bop caleb before he even bops '
                                          'ne1 plz',
                                  'child': [{'author': 'ZachityZach',
                                             'body': 'I got like 7 mins to '
                                                     "shave off before I'm "
                                                     'even competitive tho my '
                                                     'PB is 40:48 so I got a '
                                                     'lot of work to do.\n'
                       

In [ ]:
# preporcess

# remove 'deleted' text
# remove http + https links
# remove numbers

In [9]:
# Topic model LDA
# title + self text + comments
# Do Topic modeling
# Code by ... from: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for Gensim model NOT USED
# Code by ... from: https://stackabuse.com/python-for-nlp-topic-modeling/ for LDA model
#Additional resources
##https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation
##https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

import pandas as pd
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
import sys
import random

#####This is for title + selftext + comments#####
corpus = {}
corpus_two = []

def getComments(comment_list_temp):
    # get the comments
    return_list = ""
    for comment in comment_list_temp:
        return_list = return_list + comment.get('body')
    return return_list

# create the documents
for dictionary in complete_dataset:
    
    # child ( a list of dictionaries) aka our comments -- has body comment then a child comment with a body and so on
    # selftext
    # title
    title = dictionary.get('title')
    selftext = dictionary.get('selftext')
    comment_list = getComments(dictionary.get('child'))
    
    # do some cleaning here of spaces, lower textcasing etc.
    
    corpus = {
        "data": title + selftext + comment_list
    }

    corpus_two.append(corpus)

pprint(corpus_two[1])

# dump the file
with open('r_data/sub_title_selftext_comment_data.json', 'w', encoding='utf-8') as file:
    json.dump(corpus_two, file, ensure_ascii=False, indent=4)

title_selftext_comment = pd.read_json(r'r_data/sub_title_selftext_comment_data.json')
    
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(title_selftext_comment['data'].values.astype('U'))

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

LDA = LatentDirichletAllocation(n_components=15, random_state=42)
LDA.fit(doc_term_matrix)

print("DONE")


{'data': 'Tutorials for mega man games?Specifically mega man classic or the X '
         'series. a website or video would be best thanks.\n'
         'EDIT: also, leaderboards?This exists: '
         'http://megamanrta.wikia.com/wiki/Mega_Man_RTA_Wiki\n'
         '\n'
         'Though, judging from the Mega Man 1 and Rockman 2 "wr\'s", the '
         "leaderboards aren't necessarily up to date.  Have you checked the "
         '[SDA '
         "forums](https://forum.speeddemosarchive.com/board/older_consoles.html)?[deleted]Here's "
         'the [leaderboards](http://jokaah.com/mmrta/index.php) leaderboards '
         'http://jokaah.com/mmrta/\n'
         'Wiki http://megamanrta.wikia.com/wiki/Mega_Man_RTA_Wiki\n'
         '\n'
         '\n'
         'if you want to know anything about X5 or X6 you can ask me.'}
DONE


In [10]:
# Print words -- manually look at topics
first_topic = LDA.components_[0]
top_topic_words = first_topic.argsort()[-10:]

# for i in top_topic_words:
#     print(count_vect.get_feature_names()[i])

for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

print("DONE")

Top 10 words for topic #0:
['videos', 'watch', 'speedrun', 'youtube', 'http', 'tv', 'twitch', 'com', 'https', 'www']


Top 10 words for topic #1:
['sub', 'world', 'skip', 'just', 'new', 'record', 'game', 'time', 'wr', 'run']


Top 10 words for topic #2:
['http', 'twitch', 'video', 'discord', 'amp', 'capture', 'stream', 'marathon', 'com', 'https']


Top 10 words for topic #3:
['runs', 'don', 'speedrunning', 'time', 'speedrun', 'like', 'just', 'run', 'games', 'game']


Top 10 words for topic #4:
['good', 'agdq', 'don', 'time', 'gdq', 'really', 'people', 'like', 'just', 'run']


DONE


In [12]:
#USED FOR DOCUMENTS
# probability of all topics to each document
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

# Code reuse from https://nlpforhackers.io/topic-modeling/
svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(doc_term_matrix)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(title_selftext_comment))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')

# Print a plot
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [14]:
# USED FOR WORDS
# probability of all topics to each document
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

# Code reuse from https://nlpforhackers.io/topic-modeling/
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(doc_term_matrix.T)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], count_vect.get_feature_names()
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')

# Print a plot
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

In [15]:
# see the topic index with max value

#####This is for title + selftext#####
title_selftext_comment['Topic'] = topic_values.argmax(axis=1)
title_selftext_comment.head(20)

# print out spreadsheet to manually inspect?

,data,Topic
0,My new year's goal is to beat Caleb Hart to WR...,1
1,Tutorials for mega man games?Specifically mega...,0
2,[WR] My buddy just took back the WR for Mega M...,1
3,Some questions about running MGS: Twin SnakesI...,1
4,"To those of you who run multiple games, how do...",3
5,Which games saw the most changes in 2013?Which...,1
6,Interview with speedrunner CalebHart42 (Commen...,2
7,Shoutout to Joshimuz for getting sub-6 in GTA:...,1
8,Lockwood's Incredible Streets Agent 1:12 (WR) ...,1
9,Interview: The Gold Split - Feat. CalebHart42T...,4
